In [1]:
from dataclasses import dataclass
from z3 import BitVec, Optimize, Int, Solver, Sum, Optimize

In [2]:
data = open("input/10").read().splitlines()

In [3]:
@dataclass
class Machine:
    diagram: str
    wirings: list
    joltage: set

In [4]:
machines = []
for line in data:
    first_split = line.split("]")
    diagram = first_split[0][1:]

    wirings, joltage = first_split[1][1:].split("{")
    wirings = wirings.split(" ")
    formatted_wirings = []
    for wiring in wirings:
        if not wiring:
            continue
      
        wiring = eval(wiring[1:-1])
        if isinstance(wiring, int):
            wiring = (wiring,)
        formatted_wirings.append(list(wiring))
    
    joltage = joltage[:-1]
    joltage = list(map(int, joltage.split(",")))
    machines.append(Machine(diagram, formatted_wirings, joltage))

# Algorithm
Super greedy, try all possible variations and ignore early if shorter already found.
The optimized version will always be to only press each button once.

So we have 2**X possible configurations to consider. Convert that to binary and then toggle on the ones that are 1

In [5]:
def solve_one(machine):
    num_wirings = len(machine.wirings)
    target = machine.diagram
    best = num_wirings
    wanted_state = [elem == "#" for elem in machine.diagram]
    
    for i in range(2**num_wirings):
        state = [False] * len(target)
        binary = format(i, f"0{num_wirings}b")
        num_1s = 0
        for char in binary:
            if char == "1":
                num_1s += 1
        if num_1s >= best:
            continue

        for idx, char in enumerate(binary):
            if char == "1":
                cur_wiring = machine.wirings[idx]
                for w_idx in cur_wiring:
                    state[w_idx] = not state[w_idx]
        if state == wanted_state:
            best = num_1s
    return best

In [6]:
part1 = 0
for m in machines:
    part1 += solve_one(m)

In [7]:
assert part1 in (500, 7)
print(f"Answer #1: {part1}")

Answer #1: 500


# Part 2

In [8]:
def z3_me(machine):
    opt = Optimize()
    
    ops = []
    for wiring in machine.wirings:
        op = [0] * len(machine.joltage)
        for i in wiring:
            op[i] = 1
        ops.append(op)
    variables = [Int(f"x{i}") for i in range(len(ops))]
    
    for var in variables:
        opt.add(var >= 0)

    for idx in range(len(machine.joltage)):
        contribution = Sum([variables[i] * ops[i][idx] for i in range(len(ops))])
        opt.add(contribution == machine.joltage[idx])
    
    total_ops = Sum(variables)
    opt.minimize(total_ops)

    opt.check()
    model = opt.model()
    solution = [model[var].as_long() for var in variables]

    return sum(solution)
    

In [9]:
part2 = 0
for m in machines:
    part2 += z3_me(m)

In [10]:
assert part2 == 19763
print(f"Answer #2: {part2}")

Answer #2: 19763
